In [1]:
# Building and Environment
idNo = '271434'

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv

In [3]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [4]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['BIM']
# searchTerm2 = ['Sustainability']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "%20"

    fileIndex = text2.rfind("%20")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "%20"

    fileIndex = text1.rfind("%20")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green%20Building', 'Green%20Buildings', 'Sustainable%20building', 'Sustainable%20buildings', 'Environmentally%20friendly%20design', 'Sustainability', 'Sustainable%20development', 'Energy', 'Water', 'Waste', 'Thermal%20comfort', 'Electricity', 'Solar', 'Wind%20power', 'Green%20infrastructure', 'Life%20cycle%20assessment']
['BIM', 'Building%20Information%20Modelling', 'Building%20information%20model', '6D%20BIM', 'Digital%20construction', 'Smart%20construction', 'Digital%20engineering']


In [6]:
def getData(link):
    browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
    browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('span',{'class':'title-text'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title
    
    absResults = parsed_html.find('div',{'class':'abstract author'})
    
    if absResults is not None:
        try:            
            df.loc[row,'section0'] = absResults.find('h2').text
            df.loc[row,'sectionData0'] = absResults.find('div').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

        
    sections = parsed_html.find('div',{'class':'Body u-font-serif'})
    if sections is not None:        
        sections = sections.find_next('div')
        count = 1 
        for section in sections:
            if section is not None: 
                sectionNo = 'section' + str(count)
                sectionData = 'sectionData' + str(count)
                try:
                    df.loc[row,sectionNo] = section.find('h2',{'class':'u-h3 u-margin-l-top u-margin-xs-bottom'}).text
                except:
                    df.loc[row,sectionNo] = "Some issue"
                    
                data = ''
                for para in section.findAll('p'):
                    data = data + para.text
                df.loc[row,sectionData] = data
            else:
                df.loc[row,sectionNo] = "None"
            count = count + 1        

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [8]:
baseUrl = 'https://www.sciencedirect.com/search/api?tak=%22'

querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = 'https://www.sciencedirect.com/search?qs=%22' + item1 + '%22%20%2B%20%22' + item2 + '%22'
        print("base query: ",sq)
        query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
        
        # Call webdriver         
        browser = webdriver.Safari()
        browser.get(query)
        respData = browser.page_source
        browser.quit()

        # Clear html         
        respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
        respData = respData.replace('</pre></body></html>',"")
        
        py_dic = json.loads(respData)
        
        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'resultsFound':
                pageResults = py_dic[key]
        print("pageResults: ",pageResults)                
                                
        if pageResults > 25:
            page = pageResults // 25
            lastPage = pageResults % 25
            if lastPage > 0:
                page = page + 1
        else:
            page = 1
        print("page: ",page)

        #         for p in range(2):        
        for p in range(page):
            print("In page: ",p)
            if p == 0:
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
            else:
                offset = '&offset='+str(p*25)
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
                
            print("search query: ",query)  
                
            # Call webdriver         
            browser = webdriver.Safari()
            browser.get(query)
            respData = browser.page_source
            browser.quit()
            # Load Json Data 

            respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
            respData = respData.replace('</pre></body></html>',"")

            py_dic = json.loads(respData)                
            # Quit the browser 
            links = []
            for key in py_dic:
                if key == 'searchResults':
                    for searchR in py_dic[key]:
                        for search in searchR:
                            if search == 'pdf':
                                for linkR in searchR[search]:
                                    if linkR == 'getAccessLink':
                                        baseLink = 'https://www.sciencedirect.com/'
                                        link = baseLink +searchR[search][linkR]
                                        print('Reading result: ',row)
                                        df.loc[row,'searchTerm1'] = item1
                                        df.loc[row,'searchTerm2'] = item2
                                        getData(link)                                    
                                        row = row + 1        

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Building%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Building%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  0
Title:  Integration of Building Information Modeling and Web Service Application Programming Interface for assessing building surroundings in early design stages
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Buildings%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Buildings%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBs

Reading result:  24
Title:  Implementing Life Cycle Sustainability Assessment during design stages in Building Information Modelling: From systematic literature review to a methodological approach
Reading result:  25
Title:  Mixed method review for integrating building information modeling and life-cycle assessments
Reading result:  26
Title:  Mapping the scientific research of the life cycle assessment in the construction industry: A scientometric analysis
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Thermal%20comfort%22
pageResults:  2
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Thermal%20comfort%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  27
Title:  BIM- and IoT-based virtual re

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Environmentally%20friendly%20design%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Sustainability%22
pageResults:  12
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Sustainability%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  45
Title: 

Reading result:  75
Title:  A holistic framework to utilize natural ventilation to optimize energy performance of residential high-rise buildings
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Solar%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Solar%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6

Title:  Sustainability framework for buildings via data analytics
Reading result:  96
Title:  Mixed method review for integrating building information modeling and life-cycle assessments
Reading result:  97
Title:  BIM-based life cycle assessment and life cycle costing of an office building in Western Europe
Reading result:  98
Title:  Integrating building information modeling and life cycle assessment in the early and detailed building design stages
Reading result:  99
Title:  Mapping the scientific research of the life cycle assessment in the construction industry: A scientometric analysis
Reading result:  100
Title:  Developing a topological information extraction model for space syntax analysis
Reading result:  101
Title:  Ten questions concerning co-simulation for performance prediction of advanced building envelopes
Reading result:  102
Title:  Triple bottom line sustainability assessment of window-to-wall ratio in US office buildings
Reading result:  103
Title:  Application of l

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20information%20model%22%20%2B%20%22Green%20infrastructure%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B%20%22Life%20cycle%20assessment%22
pageResults:  10
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20information%20model%22%20%2B%20%22Life%20cycle%20assessment%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  123
Title:  M

base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Solar%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Solar%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Solar%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Green

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Solar%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Green%20infra

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Solar%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Wind%20power%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Wind%20power%22&show=25&publicationTitles=271434&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Green%20i

In [11]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Integration of Building Information Modeling a...,Abstract,Many studies and green building rating systems...,1. Introduction,"The Architecture, Engineering, and Constructio...",2. Implementation of components,"The integration has two main parts, Dynamo cus...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
1,BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,Integration of Building Information Modeling a...,Abstract,Many studies and green building rating systems...,1. Introduction,"The Architecture, Engineering, and Constructio...",2. Implementation of components,"The integration has two main parts, Dynamo cus...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
2,BIM,Sustainable%20building,https://www.sciencedirect.com//science/article...,Sustainable material choice for construction p...,Abstract,Construction professionals and researchers are...,1. Introduction,The construction industry is responsible for t...,2. Literature review,A literature review of the proposed methodolog...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
3,BIM,Sustainable%20buildings,https://www.sciencedirect.com//science/article...,Sustainable material choice for construction p...,Abstract,Construction professionals and researchers are...,1. Introduction,The construction industry is responsible for t...,2. Literature review,A literature review of the proposed methodolog...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
4,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Greening existing buildings through Building I...,Abstract,The use of Building Information Modelling (BIM...,1. Introduction,Building sector accounts for 36% of energy use...,2. Review approach and strategies,A systematic review was adopted to assess and ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Mapping the scientific research of the life cy...,Abstract,The interest toward the life cycle assessment ...,1. Introduction,The interest in the life cycle assessments (LC...,2. Methodology,The research methodology of this study has two...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
129,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,BIM-DLCA: An integrated dynamic environmental ...,Abstract,Life cycle assessment (LCA) is an important an...,1. Introduction,Human activities have increased carbon emissio...,2. Methodology,LCA is an important worldwide environmental ma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
130,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Life cycle assessment in the building design p...,Abstract,Buildings contribute a significant portion of ...,1. Introduction,The emerging climate crisis is driving rapid a...,2. Life cycle assessment within the building d...,LCA is an established method for assessing a w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434
131,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Triple bottom line sustainability assessment o...,Abstract,"The window-to-wall ratio (WWR), or the proport...",1. Introduction,The United Nations Environment Programme estim...,2. Methods,The large office prototype building developed ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271434


In [10]:
df1 = df
df['journalName'] = str(idNo)
fileSave = 'kgRawData/SD/sd-kg'+idNo+'.csv'
df.to_csv(fileSave)

In [1]:
idNo = '271434'

import pandas as pd
# filePath = 
data = pd.read_csv('kgRawData/SD/sd-kg' + idNo + '.csv')
data['presence'] = False

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

uniqueValidEntries = list(set(validEntries))

uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''
    
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]    
    

uniqueDf.to_csv('kgRawData/SD/cleanSd-kg'+ idNo + '.csv')

print(len(uniqueDf))

<ipython-input-1-b6aa6908d30f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


29


In [2]:
uniqueDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,Building%20information%20model,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,BIM-DLCA: An integrated dynamic environmental ...,Abstract,Life cycle assessment (LCA) is an important an...,1. Introduction,Human activities have increased carbon emissio...,2. Methodology,LCA is an important worldwide environmental ma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Sustainable%20building,https://www.sciencedirect.com//science/article...,Sustainable material choice for construction p...,Abstract,Construction professionals and researchers are...,1. Introduction,The construction industry is responsible for t...,2. Literature review,A literature review of the proposed methodolog...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Sustainable%20buildings,https://www.sciencedirect.com//science/article...,Sustainable material choice for construction p...,Abstract,Construction professionals and researchers are...,1. Introduction,The construction industry is responsible for t...,2. Literature review,A literature review of the proposed methodolog...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Implementing Life Cycle Sustainability Assessm...,Abstract,The construction sector is a major producer of...,1. Introduction,The construction sector is one of the major re...,2. Materials and methods,The research began with a SLR (Section 3) to a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Sustainable material choice for construction p...,Abstract,Construction professionals and researchers are...,1. Introduction,The construction industry is responsible for t...,2. Literature review,A literature review of the proposed methodolog...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Energy,https://www.sciencedirect.com//science/article...,A holistic framework to utilize natural ventil...,Abstract,A novel holistic framework was established usi...,1. Introduction,Mountains cover about 60% of the terrain in Ho...,2. Methodology,Fig. 1 presents the BIM-based framework for es...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BIM,Waste,https://www.sciencedirect.com//science/article...,Implementing Life Cycle Sustainability Assessm...,Abstract,The construction sector is a major producer of...,1. Introduction,The construction sector is one of the major re...,2. Materials and methods,The research began with a SLR (Section 3) to a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BIM,Electricity,https://www.sciencedirect.com//science/article...,A holistic framework to utilize natural ventil...,Abstract,A novel holistic framework was established usi...,1. Introduction,Mountains cover about 60% of the terrain in Ho...,2. Methodology,Fig. 1 presents the BIM-based framework for es...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BIM,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,BIM-DLCA: An integrated dynamic environmental ...,Abstract,Life cycle assessment (LCA) is an important an...,1. Introduction,Human activities have increased carbon emissio...,2. Methodology,LCA is an important worldwide environmental ma...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BIM,Life%20cycle%20assessment,https://www.sciencedirect.com//science/article...,Implementing Life Cycle Sustainability Assessm...,Abstract,The construction sector is a major producer of...,1. Introduction,The construction sector is one of the major re...,2. Materials and methods,The research began with a SLR (Section 3) to a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
